Stratified Sampling Data Processing Demonstration Notebook
===========================================================
This document outlines the steps involved in a the data processing for stratified sampling. It includes code snippets and explanations for calculating weights based on demographic proportions as a post-stratification technique.

# Step 0: Import libraries and load data

In [1]:
import pandas as pd
population_data = pd.read_csv('synthetic_students.csv')
# View data structure and if there is any missing values
population_data.head()

,StudentID,Gender,Age,Race,Ethnicity,StudentType,OverallGPA
0,SID000001,Male,18,White,Hispanic or Latino,Freshman,4.00
1,SID000002,Female,40,Two or more races,Hispanic or Latino,Continuing Education,0.00
2,SID000003,Female,19,White,Not Hispanic or Latino,Junior,3.23
3,SID000004,Male,20,White,Not Hispanic or Latino,Senior,3.42
4,SID000005,Female,33,Black or African American,Not Hispanic or Latino,Continuing Education,0.00


# Step 1: Identifiy the strata and its proportions in the population data

## Categorical variables

Quick overview of each categorical variable in the dataset to understand their distributions.

In [2]:
# Review each category proportions in the dataset and print the results
# Review Gender proportions
print(population_data['Gender'].value_counts(normalize=True).round(2))
print()  # Blank line for better readability
# Review Race proportions
print(population_data['Race'].value_counts(normalize=True).round(2))
print()  
# Review Ethnicity proportions
print(population_data['Ethnicity'].value_counts(normalize=True).round(2))
print()  
# Review Student Type proportions
print(population_data['StudentType'].value_counts(normalize=True).round(2))

Gender
Female     0.57
Male       0.42
Unknown    0.01
Name: proportion, dtype: float64

Race
White                                        0.55
Black or African American                    0.19
Asian                                        0.10
Two or more races                            0.07
Native Hawaiian or Other Pacific Islander    0.03
Unknown                                      0.03
American Indian or Alaska Native             0.03
Name: proportion, dtype: float64

Ethnicity
Not Hispanic or Latino    0.75
Hispanic or Latino        0.25
Name: proportion, dtype: float64

StudentType
Freshman                0.29
Senior                  0.23
Sophomore               0.19
Junior                  0.18
Continuing Education    0.10
Name: proportion, dtype: float64


## Numerical variables

Age and Overall GPA are numerical variables in the dataset and their distributions is evaluated by standard descriptive statistics.

In [3]:
# View summary statistics for numerical variables and print the results
# Overall GPA and Age
print(population_data['OverallGPA'].describe().round(2))
print()
print(population_data['Age'].describe().round(2))

count    25000.00
mean         2.81
std          1.02
min          0.00
25%          2.68
50%          3.12
75%          3.39
max          4.00
Name: OverallGPA, dtype: float64

count    25000.00
mean        19.63
std          2.82
min         17.00
25%         18.00
50%         19.00
75%         20.00
max         44.00
Name: Age, dtype: float64


We can also check the proportion of the categorical variables by the numerical variables. This will help us understand how the numerical variables are distributed across different categories. This is helpful in stratified sampling to ensure that each stratum is adequately represented in the sample because each variable will be a part of the strata and their distributions need to be known.

In [4]:
# Using student type to check what is the average Overall GPA and Age for each student type
print("Average Overall GPA by Student Type:")
print(population_data.pivot_table(values='OverallGPA', index='StudentType', aggfunc='mean').round(2))
print("Average Age by Student Type:")
print(population_data.pivot_table(values='Age', index='StudentType', aggfunc='mean').round(2))

Average Overall GPA by Student Type:
                      OverallGPA
StudentType                     
Continuing Education        0.00
Freshman                    2.99
Junior                      3.16
Senior                      3.30
Sophomore                   3.04
Average Age by Student Type:
                        Age
StudentType                
Continuing Education  25.18
Freshman              17.49
Junior                19.76
Senior                20.69
Sophomore             18.71


After the initial exploration, further steps may include data cleaning, transformation, and analysis based on the insights gathered from the initial review. This synthtetic data is already clean, so no additional cleaning steps are necessary.

# Step 2: Determine the sample size and proportions for each subgroup

This step involves calculating the required sample size from the overall population. Then, the sample size will be allocated to each stratum based on their proportions in the population.

## 2.1 Cochran's Sample Size Formula and the Finite Population Correction

In [5]:
# Cochran's Sample Size Formula and the Finite Population Correction

# Using the Scipy library stats module to get the Z value
import scipy.stats as stats

def cochran_sample_size(Z, p, e, N):
    """
    Calculate the required sample size using Cochran's formula, adjusted for a finite population.

    Parameters:
        Z (float): Z-value (e.g., 1.96 for 95% confidence level)
        Z = stats.norm.ppf(0.975)   # Z-score for the upper tail of a two-tailed 95% confidence interval (1.96 for 95% confidence)
        e (float): Margin of error (e.g., 0.05 for 5%)
        N (int): Population size

    Returns:
        int: Adjusted sample size for the finite population
    """
    n0 = (Z**2 * p * (1 - p)) / (e**2)
    n = n0 / (1 + ((n0 - 1) / N))
    return int(n)

# Parameters
Z = stats.norm.ppf(0.975)   # Calculate the critical z-value for a 95% confidence level in a standard normal distribution
p = 0.5                     # Estimated proportion (use 0.5 for maximum variability, sort of like worst-case scenario)
e = 0.05                    # Margin of error (precision level, more precision requires larger sample size)
N = 25000                   # Finite population size (in this example, student population size)

# Calculate sample size

# Calculate n0 and adjusted sample size for finite population 
n0 = (Z**2 * p * (1 - p)) / (e**2)
sample_size = n0 / (1 + ((n0 - 1) / N))
print("Cochran's Sample Size Calculation:")
print(f"Ideal sample size (n0) for 95% confidence level: {round(n0)}")
print(f"Adjusted sample size of finite population for 95% confidence level: {int(sample_size)}")

Cochran's Sample Size Calculation:
Ideal sample size (n0) for 95% confidence level: 384
Adjusted sample size of finite population for 95% confidence level: 378


## 2.2 Re-code numerical variables into categorical values for stratification

### 2.2.1 Re-code Age to Age Group

In [6]:
# Revisit the age distribution from previous exploration
print(population_data['Age'].describe().round(2))

count    25000.00
mean        19.63
std          2.82
min         17.00
25%         18.00
50%         19.00
75%         20.00
max         44.00
Name: Age, dtype: float64


From Step 1, we observed that the age distribution is right-skewed with a mean age of 19.63 years with a range from 17 to 44 years. I will demonstrate a simple method by using incremental age groups of 2 years and then using 10-year intervals for older age groups where there are fewer data points in Example 1. Then, I will demonstrate a more balanced approach using grouping that make sense for the study's context in Example 2.

**Example 1: By increments of 2 or 10 years**

In [7]:
# Binding age groups by increments of 2 years for younger ages and 10 years for older ages
population_data['Age_Group_1'] = pd.cut(population_data['Age'], 
                                       bins=[17, 19, 21, 23, 25, 27, 29, 31, 41, 50],
                                       labels=['17-18', '19-20', '21-22', '23-24', '25-26', '27-28', '29-30', '31-40', '41-50'], right=False)
# View the first 3 records to confirm the new Age_Group variable
population_data[['Age', 'Age_Group_1']].head(3)

,Age,Age_Group_1
0,18,17-18
1,40,31-40
2,19,19-20


In [8]:
# View the distribution of the new Age_Group variable
population_data['Age_Group_1'].value_counts().sort_index()

Age_Group_1
17-18    10123
19-20     8896
21-22     4222
23-24      506
25-26      329
27-28      277
29-30      235
31-40      400
41-50       12
Name: count, dtype: int64

**Example 2: By meaningful groups**

We know that in general most undergraduates will finish college between the age of 21-23, and graduate students will typically be between 24-30. Therefore, we can group ages accordingly to better reflect the population structure. In this example, there is no graduate studies but is referred as Continuing Education in the Student Type variable.

In [9]:
# Re-code Age by meaningful groups, such as typical college age, graduate age, and then beyond.
# We will keep the first three groups like the previous example, but then group all older ages into one group, as 31 and above, ' 31+

population_data['Age_Group_2'] = pd.cut(population_data['Age'], 
                                       bins=[17, 19, 21, 23, 25, 30, 44],
                                       labels=['17-18', '19-20', '21-22', '23-25', '26-30', '31+'],
                                   right=False)
# View the last 5 records to confirm the new Age_Group variable
population_data[['Age', 'Age_Group_2']].tail(5)

,Age,Age_Group_2
24995,21,21-22
24996,23,23-25
24997,20,19-20
24998,19,19-20
24999,19,19-20


In [10]:
# View the distribiution of the new Age_Group variable
population_data['Age_Group_2'].value_counts().sort_index()

Age_Group_2
17-18    10123
19-20     8896
21-22     4222
23-25      506
26-30      731
31+        521
Name: count, dtype: int64

## Step 3: Forming strata and selecting samples by proportions

A quick example to help thinking about strata formation and sample selection.

In [11]:
# How many sample cases do we need if we are to select a stratified random sample based on Age Groups?

# View sample size from the adjusted calculation
print(f"Adjusted sample size of finite population for 95% confidence level: {int(sample_size)}")

# Let's say we are to use Age_Group_2 for stratified random sampling
# The cases we need will be the proportions of each age group multiplied by the total sample size calculated earlier.
age_group_2_counts = population_data['Age_Group_2'].value_counts().sort_index()
age_group_2_proportions = age_group_2_counts / len(population_data)

# Ensure sample size is an integer since we are talking about a person, and this value comes from previous calculation
sample_size_ex = int(sample_size)  
age_group_sample_sizes = (age_group_2_proportions * sample_size_ex).round().astype(int)

# The final sample count may be more or less than the calculated sample size due to rounding.
print("Sample sizes needed for each Age Group:", age_group_sample_sizes)  

# Note. However, since we are using more than one variable for stratification, this count is only a reference point.
# We will actually be selected in the final sample will be based on the characteristics of all stratification variables combined.

Adjusted sample size of finite population for 95% confidence level: 378
Sample sizes needed for each Age Group: Age_Group_2
17-18    153
19-20    135
21-22     64
23-25      8
26-30     11
31+        8
Name: count, dtype: int64


### 3.1 More data cleaning and preparation before random sampling

#### Gender

The first variable, `Gender` has an unknown category, and we have to think about if this information is important for our analysis. Let say we want to implement some programs or initiatve after the study, but when a Gender is unknown, we may not be able to design specific services for this group without this identification. Therefore, the decision making process becomes more complex and needed careful consideration.

In [12]:
# We will make a copy of the required variables and form a new DataFrame
sample_selection_data = population_data[['StudentID', 'Gender', 'Race', 'Age_Group_2']].copy()
sample_selection_data.head()

,StudentID,Gender,Race,Age_Group_2
0,SID000001,Male,White,17-18
1,SID000002,Female,Two or more races,31+
2,SID000003,Female,White,19-20
3,SID000004,Male,White,19-20
4,SID000005,Female,Black or African American,31+


In [13]:
# For Gender, we would like to remove the 'Unknown' category and make a new column for the modified values.
sample_selection_data['Gender_m'] = sample_selection_data['Gender'].replace({'Unknown': None})

#### Race

The second variable that we are using for stratification is `Race`. Upon reviewing the race distribution, we found that some categories may be too small by proportion to produce reliable results during sampling. For example, American Indian or Alaska Native and Native Hawaiian or Other Pacific Islander by proportion are quite small. To ensure that our stratified sampling is effective and that each stratum has a sufficient number of cases for analysis, we will combine them into a single group called "Other". Again, the decision to combine categories should be made based on the research objectives and the importance of these categories in the analysis.

In [14]:
# Show the proportion of each race category from the population data
print(population_data['Race'].value_counts(normalize=True))

Race
White                                        0.55204
Black or African American                    0.18620
Asian                                        0.10100
Two or more races                            0.07016
Native Hawaiian or Other Pacific Islander    0.03088
Unknown                                      0.03068
American Indian or Alaska Native             0.02904
Name: proportion, dtype: float64


In [18]:
# For Race, we can combine 'American Indian or Alaska Native' with the 'Native Hawaiian or Other Pacific Islander', into a single 'Other' category.
sample_selection_data['Race_m'] = sample_selection_data['Race'].replace({'American Indian or Alaska Native': 'Other',
                                                          'Native Hawaiian or Other Pacific Islander': 'Other'})

# Remove Unknown from Race, Race_m is the modified column
sample_selection_data['Race_m'] = sample_selection_data['Race_m'].replace({'Unknown': None})
sample_selection_data['Race_m'].value_counts(normalize=True).round(5)

# Note. Because some cases are removed due to unknown categories and therefore the denominator changes as well.
# The proportions shown here are based on the modified dataset after removing unknowns and its different to the original population proportions.

Race_m
White                        0.56951
Black or African American    0.19209
Asian                        0.10420
Two or more races            0.07238
Other                        0.06182
Name: proportion, dtype: float64

#### Review proportions after cleaning

In [20]:
# Proportion of Gender in the Sample Selection Data
print(sample_selection_data['Gender_m'].value_counts(normalize=True).round(2))
print()
# Proportion of Race in the Sample Selection Data
print(sample_selection_data['Race_m'].value_counts(normalize=True).round(2))
print()
# Proportion of Age Group in the Sample Selection Data
print(sample_selection_data['Age_Group_2'].value_counts(normalize=True).round(2))

Gender_m
Female    0.57
Male      0.43
Name: proportion, dtype: float64

Race_m
White                        0.57
Black or African American    0.19
Asian                        0.10
Two or more races            0.07
Other                        0.06
Name: proportion, dtype: float64

Age_Group_2
17-18    0.40
19-20    0.36
21-22    0.17
26-30    0.03
31+      0.02
23-25    0.02
Name: proportion, dtype: float64


#### Other information of the dataframe

In [21]:
# What is the size of the sample selection data? 
print(f"Size of the sample selection data: {len(sample_selection_data)}")

# The dataframe didn't change, but now the variables are cleaned and ready for stratified sampling. 
# You will see missing values in the modified columns where we removed 'Unknown' categories.

# How much missing values are in each modified column?
print(sample_selection_data[['Gender_m', 'Race_m']].isnull().sum())
# What is the percentage of missing values in each modified column?
print((sample_selection_data[['Gender_m', 'Race_m']].isnull().mean() * 100).round(2))

# We have no other means to infer the missing values, so we will keep them as is for the stratified sampling process.

Size of the sample selection data: 25000
Gender_m    235
Race_m      767
dtype: int64
Gender_m    0.94
Race_m      3.07
dtype: float64


In research, transparency is key. When reporting the methodology, it is essential to provide detailed information about how the stratified sampling was conducted. This includes: noting the sample size, the stratification variables used, and the proportions of each stratum in both the population and the sample. 

Clarify any adjustments made to the data, such as combining categories or handling unknown or missing values. In this example, the 'Unknown' labels will always be there because student demographic information are often self-reported data. Therefore, it is important to explain how these cases were treated in the sampling process and the potential implications for the study's findings. You may keep the Unknown in the sampling process, but may discover that once you are forming the strata, there may not be any cases in the Unknown category selected for the sample.

You can also consider another technique called sensitivity analysis to evaluate how the assumptions made during the stratification process may impact the results. You can run the analyses with and without certain categories (e.g., including vs. excluding 'Unknown') and compare outcomes to assess robustness.

### 3.2 Calculate proportions of the strata in the sample selection data

Setup a dataframe with the combination of the three variables of each strata and then calculate their counts and proportions. 

In [22]:
# Determine the combined proportions of the strata in the sample selection data with the modified variables
stratum_set_1 = sample_selection_data.groupby(['Gender_m', 'Race_m', 'Age_Group_2'], observed=True).size().reset_index(name='Count')
stratum_set_1['Proportion'] = stratum_set_1['Count'] / stratum_set_1['Count'].sum()
stratum_set_1.head(10)

,Gender_m,Race_m,Age_Group_2,Count,Proportion
0,Female,Asian,17-18,582,0.024247
1,Female,Asian,19-20,501,0.020872
2,Female,Asian,21-22,229,0.009540
3,Female,Asian,23-25,21,0.000875
4,Female,Asian,26-30,50,0.002083
5,Female,Asian,31+,31,0.001292
6,Female,Black or African American,17-18,1066,0.044411
7,Female,Black or African American,19-20,912,0.037995
8,Female,Black or African American,21-22,462,0.019248
9,Female,Black or African American,23-25,75,0.003125


In [23]:
# In this attempt, we can see 60 strata, which is a bit wide/large for only 378 samples.
# We can forsee that some strata will have very small counts and we will face problems in pulling a sample person.

#  Quick view of the counts distribution of this stratum set.
print("The smallest stratum count is:", stratum_set_1['Count'].min())
print("The largest stratum count is:", stratum_set_1['Count'].max())

The smallest stratum count is: 14
The largest stratum count is: 3220


Note. This is a good time to explain the differences between stratified random sampling and a balanced sample approach, as it is often confused. 

A balanced sample would have equal representation from each stratum, regardless of their proportion in the population. In contrast, stratified random sampling aims to reflect the actual distribution of the population across strata, ensuring that each stratum is representing the proportion in the population. This approach helps to maintain the representativeness of the sample while still allowing for detailed analysis within each stratum. 

Often, this is the reality we face as some groups are less representative when compared to the overall population. There are other techniques to address this issue, such as oversampling underrepresented strata or using different sampling methods altogether. It is important to consider these factors when designing a study and interpreting the results. 

There are also post-hoc techniques to adjust for small strata during analysis, such as weighting or combining strata when the results are reported. There will be a discussion of these techniques at the end of today's presentation.

In [24]:
# Let's review the stratum counts that have less than 30 cases and it means that within this combination, there are this many students to be randomly selected from.
stratum_set_1[stratum_set_1['Count'] < 30]

,Gender_m,Race_m,Age_Group_2,Count,Proportion
3,Female,Asian,23-25,21,0.000875
15,Female,Other,23-25,16,0.000667
16,Female,Other,26-30,27,0.001125
17,Female,Other,31+,19,0.000792
21,Female,Two or more races,23-25,19,0.000792
22,Female,Two or more races,26-30,28,0.001167
23,Female,Two or more races,31+,24,0.001000
33,Male,Asian,23-25,18,0.000750
34,Male,Asian,26-30,23,0.000958
35,Male,Asian,31+,26,0.001083


From the output, we can see that most of the small strata are associated with age groups that are older than typical college age and the race groups with smaller representation in the population. For demonstration purposes, we will continue to next steps so that you can see how samples are drawn from the strata set 1. Then, we will discuss other options. 

### 3.3. Draw the random sample based on the strata proportions in set 1

In [25]:
# Find the updated stratum sizes by using the proportions time the calculated sample size (n = 378)
stratum_sizes_updated = (stratum_set_1['Proportion'] * sample_size_ex).round().astype(int)
# Add the grouping variables back to the stratum sizes DataFrame for clarity
stratum_sizes_updated = pd.concat([stratum_set_1[['Gender_m', 'Race_m', 'Age_Group_2']], stratum_sizes_updated.rename('Sample_Size')], axis=1)
stratum_sizes_updated.head()

,Gender_m,Race_m,Age_Group_2,Sample_Size
0,Female,Asian,17-18,9
1,Female,Asian,19-20,8
2,Female,Asian,21-22,4
3,Female,Asian,23-25,0
4,Female,Asian,26-30,1


As you can see in this example, the needed sample sizes for some strata are 1 or less than 1, which is not feasible for sampling.

At this stage, any proportional adjustments can be made to ensure that no stratum has a sample size less than 1. If we do that though, we consider it "disproportionate stratified sampling" rather than a "proportionate stratified sampling" approach. Because you sort of "over-sample" some strata to ensure they have at least one sample. This can be done if your objective is to increase precision for key subgroups. 

Another method is to increase the overall sample size, for example the current sample size estimate is using 95% confidence intervals and 5% margin of error. If we tightened the margin of error to 1% or 3%, the overall sample size will increase, which may help address the small stratum sizes. However, this approach also has trade-offs, as a larger sample size may require more resources and there is no guarantee that all strata will have sufficient representation during the survey administration.

## Repeat Step 2 and estimate a larger sample size

#### Example 1: Reduce margin of error from 5% to 3%

In [26]:
# This time we are using a different set of parameters. The formula remains the same from Step 2.

# Parameters
Z = stats.norm.ppf(0.975)   # Calculate the critical z-value for
# a 95% confidence level in a standard normal distribution
p = 0.5                     # Estimated proportion (use 0.5 for maximum variability, sort of like worst-case scenario)
e_3 = 0.03                    # Margin of error (precision level, more precision requires larger sample size)
N = 25000                   # Finite population size (in this example, student population size)

# Calculate sample size
# Calculate n0 and adjusted sample size for finite population
n0 = (Z**2 * p * (1 - p)) / (e_3**2)
sample_size_larger_e = n0 / (1 + ((n0 - 1) / N))
print(f"Ideal sample size (n0) for 95% confidence level with 3% margin of error: {round(n0)}")
print(f"Adjusted sample size of finite population for 95% confidence level with 3% margin of error: {int(sample_size_larger_e)}")

Ideal sample size (n0) for 95% confidence level with 3% margin of error: 1067
Adjusted sample size of finite population for 95% confidence level with 3% margin of error: 1023


#### Example 2: Reduce margin of error from 5% to 1%

In [27]:
# What if the margin of error is reduced to 1%?
# Calculate sample size
e_1 = 0.01                    # Margin of error (precision level, more precision requires larger sample size)
# Calculate n0 and adjusted sample size for finite population
n0 = (Z**2 * p * (1 - p)) / (e_1**2)
sample_size_smaller_e = n0 / (1 + ((n0 - 1) / N))
print(f"Ideal sample size (n0) for 95% confidence level with 1% margin of error: {round(n0)}")
print(f"Adjusted sample size of finite population for 95% confidence level with 1% margin of error: {int(sample_size_smaller_e)}")

Ideal sample size (n0) for 95% confidence level with 1% margin of error: 9604
Adjusted sample size of finite population for 95% confidence level with 1% margin of error: 6938


#### Example 3: Increase confidence level from 95% to 99% and keep margin of error at 5%

In [28]:
# Will increasing the confidence interval to 99% help?
# Calculate sample size
Z = stats.norm.ppf(0.995)   # Calculate the critical z-value for a 99% confidence level in a standard normal distribution
p = 0.5                     # Estimated proportion (use 0.5 for maximum variability
e_5 = 0.05                    # Margin of error (precision level, more precision requires larger sample size)
N = 25000                   # Finite population size (in this example, student population size

# Calculate sample size
# Calculate n0 and adjusted sample size for finite population
n0 = (Z**2 * p * (1 - p)) / (e_5**2)
sample_size_99ci = n0 / (1 + ((n0 - 1) / N))
print(f"Ideal sample size (n0) for 99% confidence level: {round(n0)}")
print(f"Adjusted sample size of finite population for 99% confidence level: {int(sample_size_99ci)}")

Ideal sample size (n0) for 99% confidence level: 663
Adjusted sample size of finite population for 99% confidence level: 646


Decisions on which sample size will be the best to use and how it aligns with the research objectives will need to be made.

### 3.4 Draw the random sample based on the strata proportions in set 1 and with the sample size estimates with 3% margin of error and 99% confidence level

In [29]:
# use the 3% sample size to draw the stratified random sample
stratum_sizes_three_moe = (stratum_set_1['Proportion'] * int(sample_size_larger_e)).round().astype(int)
# Add the grouping variables back to the stratum sizes DataFrame for clarity
stratum_sizes_three_moe = pd.concat([stratum_set_1[['Gender_m', 'Race_m', 'Age_Group_2']], stratum_sizes_three_moe.rename('Sample_Size')], axis=1)
stratum_sizes_three_moe.head()

,Gender_m,Race_m,Age_Group_2,Sample_Size
0,Female,Asian,17-18,25
1,Female,Asian,19-20,21
2,Female,Asian,21-22,10
3,Female,Asian,23-25,1
4,Female,Asian,26-30,2


A quick comparison since the sample size count still shows 1 person or 2 people for some strata.

In [30]:
# use the 99% confidence level sample size to draw the stratified random sample
stratum_sizes_99ci = (stratum_set_1['Proportion'] * int(sample_size_99ci)).round().astype(int)
# Add the grouping variables back to the stratum sizes DataFrame for clarity
stratum_sizes_99ci = pd.concat([stratum_set_1[['Gender_m', 'Race_m', 'Age_Group_2']], stratum_sizes_99ci.rename('Sample_Size')], axis=1)
stratum_sizes_99ci.head()

,Gender_m,Race_m,Age_Group_2,Sample_Size
0,Female,Asian,17-18,16
1,Female,Asian,19-20,13
2,Female,Asian,21-22,6
3,Female,Asian,23-25,1
4,Female,Asian,26-30,1


After some iterations, let say we decided to go with the 3% margin of error sample size estimate. We will then proceed to draw the random sample from the dataset.

## Step 4. Apply random selection and form the completed sample dataset

In [31]:
# Initialize an empty list to hold sampled dataframes
sampled_groups = []
for i, row in stratum_sizes_three_moe.iterrows():
    stratum_filter = (
        (sample_selection_data['Gender_m'] == row['Gender_m']) &
        (sample_selection_data['Race_m'] == row['Race_m']) &
        (sample_selection_data['Age_Group_2'] == row['Age_Group_2'])
    )
    stratum_population = sample_selection_data[stratum_filter]
    # Use the sample size from the current row
    stratum_sample_size = int(row['Sample_Size'])

    # Ensure we don't sample from empty strata or more than available
    n = min(stratum_sample_size, len(stratum_population))
    if n > 0:
        stratum_sample = stratum_population.sample(n=n, replace=False)
        sampled_groups.append(stratum_sample)

# Combine all sampled dataframes into a single dataframe (guard against empty list)
if sampled_groups:
    sampled_dataset = pd.concat(sampled_groups).reset_index(drop=True)
else:
    sampled_dataset = pd.DataFrame(columns=sample_selection_data.columns)

# Create an alias expected by downstream cells and align column naming
sampled_data = sampled_dataset.copy()
if 'Age_Group' not in sampled_data.columns and 'Age_Group_2' in sampled_data.columns:
    sampled_data['Age_Group'] = sampled_data['Age_Group_2']

sampled_dataset.head()

,StudentID,Gender,Race,Age_Group_2,Gender_m,Race_m
0,SID000860,Female,Asian,17-18,Female,Asian
1,SID003994,Female,Asian,17-18,Female,Asian
2,SID005995,Female,Asian,17-18,Female,Asian
3,SID003674,Female,Asian,17-18,Female,Asian
4,SID011387,Female,Asian,17-18,Female,Asian


This roster will be the final sample dataset that can be used for survey administration, and the system we used allowed us to embed unique identifiers to track responses back to the sample roster while maintaining respondent anonymity. In your process, you may have to consider other methods to ensure anonymity while still being able to link responses to the sample roster for analysis purposes.

### Last Check: Proportions in final sample dataset and population dataset

#### Sample dataset proportions

In [ ]:
# Calculate the proportions in the sampled data
sample_proportions = sampled_dataset.groupby(['Gender_m', 'Race_m', 'Age_Group_2'], observed=True).size().reset_index(name='Count')
sample_proportions['Proportion'] = sample_proportions['Count'] / sample_proportions['Count'].sum()
sample_proportions.head(10)
# What you want to do is compare the 'Proportion' column in sample_proportions with the population proportions.

,Gender_m,Race_m,Age_Group_2,Count,Proportion
0,Female,Asian,17-18,25,0.024343
1,Female,Asian,19-20,21,0.020448
2,Female,Asian,21-22,10,0.009737
3,Female,Asian,23-25,1,0.000974
4,Female,Asian,26-30,2,0.001947
5,Female,Asian,31+,1,0.000974
6,Female,Black or African American,17-18,45,0.043817
7,Female,Black or African American,19-20,39,0.037975
8,Female,Black or African American,21-22,20,0.019474
9,Female,Black or African American,23-25,3,0.002921


#### Population dataset proportions

In [33]:
# Calculate the proportions in the population data
population_proportions = sample_selection_data.groupby(['Gender_m', 'Race_m', 'Age_Group_2'], observed=True).size().reset_index(name='Count')
population_proportions['Proportion'] = population_proportions['Count'] / population_proportions['Count'].sum()
population_proportions.head(10)

,Gender_m,Race_m,Age_Group_2,Count,Proportion
0,Female,Asian,17-18,582,0.024247
1,Female,Asian,19-20,501,0.020872
2,Female,Asian,21-22,229,0.009540
3,Female,Asian,23-25,21,0.000875
4,Female,Asian,26-30,50,0.002083
5,Female,Asian,31+,31,0.001292
6,Female,Black or African American,17-18,1066,0.044411
7,Female,Black or African American,19-20,912,0.037995
8,Female,Black or African American,21-22,462,0.019248
9,Female,Black or African American,23-25,75,0.003125


Notice that it will not be exactly the same proportions because of the small sample sizes in some strata, but they should be very close. If there are significant differences, it may indicate issues in the sampling process that need to be addressed.

# Weight calculation during post-stratification

Survey weights can also be calculated by different methods. This demonstration shows the most common way to calculate weights using the original proportion from sampling and then the proportion obtained fromt he respondents data.

In [34]:
# Import the respondents data file (synthetic respondents data based on a survey)
respondents_data = pd.read_excel('respondents_file.xlsx')
respondents_data.head()

,Q1_Gender,Q2_Race,Q3_AgeGroup,Q4_Traffic_Accidents,Q5_Academic_Performance
0,Male,White,17-18,4.0,4.0
1,Female,White,17-18,3.0,3.0
2,Female,White,17-18,5.0,5.0
3,Female,Asian,17-18,5.0,5.0
4,Female,Black or African American,17-18,5.0,5.0


Weighting: The use of marginal weights (raking) are meant to adjust for differences in the marginal distributions, which is a common post-stratification technique, especially if some joint cells are sparse or empty. This involves adjusting the weights so that the weighted distribution of the sample matches the known population distribution for certain key variables.

In [35]:
# Recall the propotions from the sampled data
sampled_data_proportions = {
    "gender_sampled": sampled_dataset['Gender_m'].value_counts(normalize=True).round(3).to_dict(),
    "race_sampled": sampled_dataset['Race_m'].value_counts(normalize=True).round(3).to_dict(),
    "age_group_sampled": sampled_dataset['Age_Group_2'].value_counts(normalize=True).round(3).to_dict()
}
sampled_data_proportions

{'gender_sampled': {'Female': 0.574, 'Male': 0.426},
 'race_sampled': {'White': 0.568,
  'Black or African American': 0.192,
  'Asian': 0.103,
  'Two or more races': 0.074,
  'Other': 0.063},
 'age_group_sampled': {'17-18': 0.404,
  '19-20': 0.354,
  '21-22': 0.168,
  '26-30': 0.029,
  '31+': 0.022,
  '23-25': 0.021}}

In [36]:
## Calculate the respondents demographic proportions of the three variables: Gender, Race, and Age Group and create a dictionary by the propotion and round to three decimal places
respondent_proportions = {
    "Gender_r": respondents_data['Q1_Gender'].value_counts(normalize=True).round(3).to_dict(),
    "Race_r": respondents_data['Q2_Race'].value_counts(normalize=True).round(3).to_dict(),
    "Age_Group_r": respondents_data['Q3_AgeGroup'].value_counts(normalize=True).round(3).to_dict()
}
respondent_proportions

{'Gender_r': {'Female': 0.631, 'Male': 0.369},
 'Race_r': {'White': 0.556,
  'Black or African American': 0.162,
  'Asian': 0.154,
  'Two or more races': 0.083,
  'Other': 0.046},
 'Age_Group_r': {'19-20': 0.419,
  '17-18': 0.261,
  '21-22': 0.133,
  '23-25': 0.095,
  '31+': 0.091}}

We noticed there are some differences between the samples demographic proportions and the respondents demographic proportions. To adjust for these differences during the analysis phase, we can calculate weights for each demographic category based on the ratio of the respondent proportions to the sampled data proportions. These weights can then be applied during the analysis to ensure that the results accurately reflect the population demographics.

## Create the weight calculation function based on the demographic proportions

In [38]:
# Define the two dictionaries for population and respondents proportions
popl_dict = sampled_data_proportions
resp_dict = respondent_proportions

In [39]:
# Using each of the labels in the variables, then the population or respondent proportions can be retrieved from the dictionaries.
# For example, if Q1_Gender is 'Female', then the population proportion of 0.574 will be retrieved.
# Then, the respondent proportion, 'Gender_r' of 0.631 will be retrieved for the denominator.

# Now we can calculate the weights for each respondent based on the three variables
def calculate_weight(row, popl_dict, resp_dict):
    gender_weight = popl_dict['gender_sampled'].get(row['Q1_Gender'], 0) / resp_dict['Gender_r'].get(row['Q1_Gender'], 1)
    race_weight = popl_dict['race_sampled'].get(row['Q2_Race'], 0) / resp_dict['Race_r'].get(row['Q2_Race'], 1)
    age_group_weight = popl_dict['age_group_sampled'].get(row['Q3_AgeGroup'], 0) / resp_dict['Age_Group_r'].get(row['Q3_AgeGroup'], 1)
    total_weight = gender_weight * race_weight * age_group_weight
    return total_weight

# Apply the weight calculation to each row in the respondents_data DataFrame
respondents_data['Weight'] = respondents_data.apply(calculate_weight, axis=1, args=(popl_dict, resp_dict))
respondents_data

,Q1_Gender,Q2_Race,Q3_AgeGroup,Q4_Traffic_Accidents,Q5_Academic_Performance,Weight
0,Male,White,17-18,4.0,4.0,1.825566
1,Female,White,17-18,3.0,3.0,1.438457
2,Female,White,17-18,5.0,5.0,1.438457
3,Female,Asian,17-18,5.0,5.0,0.941759
4,Female,Black or African American,17-18,5.0,5.0,1.668820
...,...,...,...,...,...,...
236,Female,Two or more races,17-18,NaN,NaN,1.255385
237,Male,Asian,17-18,NaN,NaN,1.195200
238,Male,White,23-25,3.0,3.0,0.260707
239,Female,Two or more races,17-18,NaN,NaN,1.255385


The above function calculates weights for each demographic category based on the ratio of the respondent proportions to the sampled data proportions. These weights can then be applied during the analysis to ensure that the results accurately reflect the population demographics.
1. Accessing a dictionary inside another dictionary. For example, 'Gender_r' is a key in resp_dict, and its value was determined earlier from the respondent proportions calculation.

2. Looking up a value. Then you see the use of get() method to look up the value corresponding to the key row['Q1_Gender'] from the respondents survey result dataframe. 
This step will use the recorded value in the survey results dataframe to find the corresponding proportion in both the population and respondents dictionaries.

3. Lastly, the weight for each demographic category is calculated by dividing the population proportion by the respondent proportion. The total weight for each respondent is then obtained by multiplying the individual weights together.